# Problem 3
Powerco has three electric power plants that supply the needs of four cities. Each power plan can supply the following numbers of kilowatt-hours (kwh) of electricity (cf. Table below):
* plant 1: 35 million
* plant 2: 50 million
* plant 3: 40 million

The peak power demands in these cities, which occur at the same time (2PM), are as follows (in kwh):
* Lausanne: 45 million
* Geneva: 20 million
* Sion: 30 million
* Bulle: 30 million

The costs of sending 1 million kwh of electricity from plant to city depend on the distance the electricity must travail.

Formulate an LP to minimize the cost of meeting each city's peak power demand.

| From / To | Lausanne | Geneva | Sion | Bulle | Supply in million kwh |
| - | - | - | - | - | - |
| Plant 1 | $8 | $6 | $10 | $9 | 35 |
| Plant 2 | $9 | $12 | $13 | $7 | 50 |
| Plant 3 | $14 | $9 | $16 | $5 | 40 |
| Demand in million kwh | 45 | 20 | 30 | 30 | 125 / 125 |

## Problem formulation
$x_{pc}$ number of million kwh produced at plant $p$ and sent to city $c$.

With
* $p \in \{1,2,3\}$
* $c \in \{1,2,3,4\}$

So, we want to

$$ \min{\kappa_1} = 8x_{11} + 6x_{12} + 10x_{13} + 9 x_{14} $$

Where $\kappa_1$ is the cost of plant 1.

As we want to take into account all plants, our objective function develops as follows:
$$ \min{\Kappa} = (8x_{11} + 6x_{12} + 10x_{13} + 9 x_{14}) + (9x_{21} + 12x_{22} + 13x_{23} + 7x_{24}) + (14x_{31} + 9x_{32} + 16x_{33} + 5x_{34})$$
$$ \min{\Kappa} = \sum_{c \in C} \sum_{p \in P} \kappa_{pc} \times x_{pc} $$

Where $\kappa_{pc}$ is the cost of transmission of plant $p$ to city $c$.

The supply constraints are:
$$ x_{11} + x_{12} + x_{13} + x_{14} \leq 35$$
$$ x_{21} + x_{22} + x_{23} + x_{24} \leq 50$$
$$ x_{31} + x_{32} + x_{33} + x_{34} \leq 40$$


The demand constraints are:
$$ x_{11} + x_{21} + x_{31} \geq 45 $$
$$ x_{12} + x_{22} + x_{32} \geq 20 $$
$$ x_{13} + x_{23} + x_{33} \geq 30 $$
$$ x_{14} + x_{24} + x_{34} \geq 30 $$


The decision variable should be
$$ x_{pc} \geq 0 \, (p=1,2,3;\, c=1,2,3,4) $$

### Elegant notation
$$ \min{\Kappa} = \sum_{c}^C \sum_{p}^P \kappa_{pc} \times x_{pc} $$
$$ s.t. $$
$$ \sum_{c \in C}^{C=4} x_{1c} \leq S_1,\quad \sum_{c \in C}^{C=4} x_{2c} \leq S_2,\quad \sum_{c \in C}^{C=4} x_{3c} \leq S_3 \quad \forall p \in P $$
$$ \sum_{p}^{P=3} x_{p1} \geq D_1,\quad \sum_{p}^{P=3} x_{p2} \geq D_2,\quad \sum_{p}^{P=3} x_{p3} \geq D_3,\quad \sum_{p}^{P=3} x_{p4} \geq D_4 \quad \forall c \in C $$

Where $S_p$ is the supply $S$ of plant $p$ and $D_c$ is the demand $D$ of city $c$.


### Matrix notation
$$ \min \Kappa_{[3 \times 4]} \mathbf{X}^\top_{[4 \times 3]}  $$
$$ s.t. $$
$$ \mathbf{X}_{[3 \times 4]} \mathbf{1}_{[4 \times 1]} \leq \overrightarrow{s}^{\top}_{[3 \times 1]}, \qquad \overrightarrow{s} = \begin{bmatrix} S_1 & S_2 & S_3 \end{bmatrix} $$
$$  \mathbf{X}^\top_{[4 \times 3]} \mathbf{1}_{[3 \times 1]} \geq \overrightarrow{d}^{\top}_{[4 \times 1]}, \qquad \overrightarrow{d} = \begin{bmatrix} D_1 & D_2 & D_3 & D_4 \end{bmatrix}$$

Where

$$ \mathbf{X} = 
 \begin{pmatrix}
x_{11} & x_{12} & x_{13} & x_{14} \\
x_{21} & x_{22} & x_{23} & x_{24} \\
x_{31} & x_{32} & x_{33} & x_{34}
 \end{pmatrix} $$

And

$\Kappa$ the matrix of costs of transmission.

In [1]:
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [2]:
# Cost matrix (or Kappa)
C = np.array([[8, 6, 10, 9],
              [9, 12, 13, 7],
              [14, 9, 16, 5]])

# Max supply
S = np.array([35, 50, 40])

# Min demand
D = np.array([45, 20, 30, 30])

In [3]:
supply = dict(zip(range(1, 4), S))
print(f"Supply per plant: {supply}")

demand = dict(zip(range(1, 5), D))
print(f"Demand per city: {demand}")

cost_idx = [(p, c) for p in range(1, 4) for c in range(1, 5)]
cost_numpy_idx = [(p, c) for p in range(3) for c in range(4)]
print(f"The Pyomo indices to initialize the Costs Parameter: {cost_idx}")
print(f"The NumPy indices to access each cost in the matrix: {cost_numpy_idx}")

Supply per plant: {1: 35, 2: 50, 3: 40}
Demand per city: {1: 45, 2: 20, 3: 30, 4: 30}
The Pyomo indices to initialize the Costs Parameter: [(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4)]
The NumPy indices to access each cost in the matrix: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]


In [4]:
init_cost_param = {cost_idx[i]: C[c] for i, c in enumerate(cost_numpy_idx)}
init_cost_param

{(1, 1): 8,
 (1, 2): 6,
 (1, 3): 10,
 (1, 4): 9,
 (2, 1): 9,
 (2, 2): 12,
 (2, 3): 13,
 (2, 4): 7,
 (3, 1): 14,
 (3, 2): 9,
 (3, 3): 16,
 (3, 4): 5}

In [7]:
# The model
model = pyo.ConcreteModel()

# Sets
model.cities = pyo.RangeSet(1, 4)
model.plants = pyo.RangeSet(1, 3)

# Parameters
model.Supply = pyo.Param(model.plants, initialize=supply)  # {1: 35, 2: 50, 3: 40}
model.Demand = pyo.Param(model.cities, initialize=demand)  # {1: 45, 2: 20, 3: 30, 4: 30}
model.Costs = pyo.Param(model.plants, model.cities, initialize=init_cost_param)  # Cost of transmission from plants to cities

# Decision variables
model.x = pyo.Var(model.plants, model.cities, within=pyo.NonNegativeReals)

# Objective function
def objective_callback(model: pyo.Model):
    return sum(sum(model.Costs[p, c] * model.x[p, c] for p in model.plants) for c in model.cities)


model.ObjectiveFunction = pyo.Objective(rule=objective_callback, sense=pyo.minimize)

# Constraints
def supply_constraint_1(model, p):
    return sum(model.x[1, c] for c in model.cities) <= model.Supply[1]


model.ConstraintSupply1 = pyo.Constraint(model.cities, rule=supply_constraint_1)


def supply_constraint_2(model, p):
    return sum(model.x[2, c] for c in model.cities) <= model.Supply[2]


model.ConstraintSupply2 = pyo.Constraint(model.cities, rule=supply_constraint_2)


def supply_constraint_3(model, p):
    return sum(model.x[3, c] for c in model.cities) <= model.Supply[3]


model.ConstraintSupply3 = pyo.Constraint(model.cities, rule=supply_constraint_3)


def demand_constraint_1(model, c):
    return sum(model.x[p, 1] for p in model.plants) >= model.Demand[1]


model.ConstraintDemand1 = pyo.Constraint(model.plants, rule=demand_constraint_1)


def demand_constraint_2(model, c):
    return sum(model.x[p, 2] for p in model.plants) >= model.Demand[2]


model.ConstraintDemand2 = pyo.Constraint(model.plants, rule=demand_constraint_2)


def demand_constraint_3(model, c):
    return sum(model.x[p, 3] for p in model.plants) >= model.Demand[3]


model.ConstraintDemand3 = pyo.Constraint(model.plants, rule=demand_constraint_3)


def demand_constraint_4(model, c):
    return sum(model.x[p, 4] for p in model.plants) >= model.Demand[4]


model.ConstraintDemand4 = pyo.Constraint(model.plants, rule=demand_constraint_4)

# Solve the model
solver = SolverFactory("glpk")
results = solver.solve(model)
print(f"Objective function =  {model.ObjectiveFunction()}")

for p in model.plants:
    for c in model.cities:
        print(f"Electricity sent from plant {p} to city {c} = {model.x[p, c]()}")

Objective function =  0.0
Electricity sent from plant 1 to city 1 = 0.0
Electricity sent from plant 1 to city 2 = 0.0
Electricity sent from plant 1 to city 3 = 0.0
Electricity sent from plant 1 to city 4 = 0.0
Electricity sent from plant 2 to city 1 = 0.0
Electricity sent from plant 2 to city 2 = 0.0
Electricity sent from plant 2 to city 3 = 0.0
Electricity sent from plant 2 to city 4 = 0.0
Electricity sent from plant 3 to city 1 = 0.0
Electricity sent from plant 3 to city 2 = 0.0
Electricity sent from plant 3 to city 3 = 0.0
Electricity sent from plant 3 to city 4 = 0.0
